In [ ]:
import numpy as np
import pandas as pd

n_observations = 1200 
n_features = 4        

np.random.seed(42)
base_series = np.cumsum(np.random.normal(0, 1, n_observations))

data_dict = {'feature_1': base_series}
for i in range(1, n_features):
    noise = np.random.normal(0, 0.5, n_observations)
    correlated_series = base_series + noise * (1 + i * 0.1)
    data_dict[f'feature_{i+1}'] = correlated_series

df = pd.DataFrame(data_dict)

print(f"Dataset shape: {df.shape}")
print("First 5 rows of the dataset:")
print(df.head())

observations_met = df.shape[0] >= 1000
features_met = df.shape[1] >= 3

print(f"Observations >= 1000: {observations_met}")
print(f"Features >= 3: {features_met}")

from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()

df_scaled = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)
print("First 5 rows of scaled dataset:")
print(df_scaled.head())

df_differenced = df_scaled.diff().dropna()
print("\nFirst 5 rows of differenced dataset:")
print(df_differenced.head())

print(f"\nShape of differenced dataset: {df_differenced.shape}")

import numpy as np

def create_sequences(data, look_back):
    X, y = [], []
    for i in range(len(data) - look_back):
        X.append(data[i:(i + look_back)])
        y.append(data[i + look_back])
    return np.array(X), np.array(y)

look_back = 20

data_array = df_differenced.values

X, y = create_sequences(data_array, look_back)

print(f"Shape of X (sequences): {X.shape}")
print(f"Shape of y (targets): {y.shape}")

from sklearn.model_selection import train_test_split

train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

total_samples = len(X)
train_samples = int(total_samples * train_ratio)
val_samples = int(total_samples * val_ratio)

X_train = X[:train_samples]
y_train = y[:train_samples]

X_val = X[train_samples : train_samples + val_samples]
y_val = y[train_samples : train_samples + val_samples]

X_test = X[train_samples + val_samples :]
y_test = y[train_samples + val_samples :]

print(f"X_train shape: {X_train.shape}, y_train shape: {y_train.shape}")
print(f"X_val shape: {X_val.shape}, y_val shape: {y_val.shape}")
print(f"X_test shape: {X_test.shape}, y_test shape: {y_test.shape}")

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

print("TensorFlow and Keras components imported successfully.")

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense

n_features = X_train.shape[2] 

model_lstm = Sequential([
    tf.keras.Input(shape=(look_back, n_features)), 
    LSTM(units=50, activation='relu'),
    Dense(units=n_features)
])

model_lstm.compile(optimizer='adam', loss='mse')

print(model_lstm.summary())

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense
from tensorflow.keras.models import Model

print("Keras functional API components imported successfully.")

encoder_units = 64

encoder_inputs = Input(shape=(look_back, n_features))
encoder_lstm = LSTM(encoder_units, return_sequences=True, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
encoder_states = [state_h, state_c]

decoder_inputs = Input(shape=(1, n_features)) 
decoder_lstm = LSTM(encoder_units, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)

encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(encoder_units,))
decoder_state_input_c = Input(shape=(encoder_units,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h_decoder, state_c_decoder = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs
)
decoder_states = [state_h_decoder, state_c_decoder]

decoder_dense = Dense(n_features)
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states
)

print("Encoder Model Summary:")
print(encoder_model.summary())

print("\nDecoder Model Summary:")
print(decoder_model.summary())

import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, AdditiveAttention, Concatenate
from tensorflow.keras.models import Model

decoder_inputs_attn = Input(shape=(1, n_features), name='decoder_input_attention') 
decoder_state_input_h = Input(shape=(encoder_units,), name='decoder_state_input_h')
decoder_state_input_c = Input(shape=(encoder_units,), name='decoder_state_input_c')
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_lstm_layer = LSTM(encoder_units, return_sequences=True, return_state=True, name='decoder_lstm_layer')
decoder_lstm_outputs, decoder_state_h, decoder_state_c = decoder_lstm_layer(
    decoder_inputs_attn, initial_state=decoder_states_inputs
)

attention_layer = AdditiveAttention(name='bahdanau_attention')
attention_output = attention_layer([decoder_lstm_outputs, encoder_outputs])

decoder_concat_input = Concatenate(axis=-1, name='decoder_output_and_attention')([decoder_lstm_outputs, attention_output])

decoder_dense_layer = Dense(n_features, name='decoder_output_dense_layer')
decoder_outputs = decoder_dense_layer(decoder_concat_input)

model_attention_seq2seq = Model(
    [encoder_inputs, decoder_inputs_attn, decoder_state_input_h, decoder_state_input_c],
    [decoder_outputs, decoder_state_h, decoder_state_c]
)

model_attention_seq2seq.compile(optimizer='adam', loss='mse')

print("\nAttention-based Seq2Seq Model Summary:")
print(model_attention_seq2seq.summary())

epochs = 50
batch_size = 32

print("\nTraining Baseline LSTM Model...")
history_lstm = model_lstm.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_val, y_val),
    verbose=1
)

print("Baseline LSTM Model Training Complete.")

epochs = 50
batch_size = 32 

print("\nPreparing inputs for Attention-based Seq2Seq Model...")

encoder_h_train, encoder_c_train = encoder_model.predict(X_train)

encoder_h_val, encoder_c_val = encoder_model.predict(X_val)

decoder_input_train = y_train.reshape(y_train.shape[0], 1, y_train.shape[1])

decoder_input_val = y_val.reshape(y_val.shape[0], 1, y_val.shape[1])

print(f"encoder_h_train shape: {encoder_h_train.shape}")
print(f"encoder_c_train shape: {encoder_c_train.shape}")
print(f"decoder_input_train shape: {decoder_input_train.shape}")

print("Inputs preparation complete.")

print("\nTraining Attention-based Seq2Seq Model...")

history_attention = model_attention_seq2seq.fit(
    [X_train, decoder_input_train, encoder_h_train, encoder_c_train],
    [y_train, encoder_h_train, encoder_c_train], 
    epochs=epochs,
    batch_size=batch_size,
    validation_data=([
        X_val, decoder_input_val, encoder_h_val, encoder_c_val
    ],
    [y_val, encoder_h_val, encoder_c_val]
    ),
    verbose=1
)

print("Attention-based Seq2Seq Model Training Complete.")

import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, LSTM, Dense, AdditiveAttention, Concatenate
from tensorflow.keras.models import Model

print("Generating predictions for Baseline LSTM Model...")
lstm_predictions = model_lstm.predict(X_test, verbose=0)
print("Baseline LSTM Model predictions generated.")
print("Setting up inference models for Attention-based Seq2Seq Model...")

encoder_inputs_inf = Input(shape=(look_back, n_features), name='encoder_inputs_inference_graph')
encoder_outputs_inf, state_h_inf, state_c_inf = encoder_lstm(encoder_inputs_inf) 
encoder_inference_model = Model(encoder_inputs_inf, [encoder_outputs_inf, state_h_inf, state_c_inf])
decoder_input_single_step = Input(shape=(1, n_features), name='decoder_input_single_step')
decoder_h_input = Input(shape=(encoder_units,), name='decoder_h_input')
decoder_c_input = Input(shape=(encoder_units,), name='decoder_c_input')
decoder_encoder_outputs_input = Input(shape=(look_back, encoder_units), name='decoder_encoder_outputs_input') 

decoder_lstm_outputs_inf, new_h, new_c = decoder_lstm_layer(
    decoder_input_single_step, initial_state=[decoder_h_input, decoder_c_input]
)

attention_output_inf = attention_layer([decoder_lstm_outputs_inf, decoder_encoder_outputs_input])
decoder_concat_input_inf = Concatenate(axis=-1, name='decoder_output_and_attention_inference')([decoder_lstm_outputs_inf, attention_output_inf])
decoder_outputs_inf = decoder_dense_layer(decoder_concat_input_inf)

decoder_inference_model = Model(
    [decoder_input_single_step, decoder_h_input, decoder_c_input, decoder_encoder_outputs_input],
    [decoder_outputs_inf, new_h, new_c]
)

print("Attention-based Seq2Seq Inference models created.")

attention_predictions = []

print("Generating predictions for Attention-based Seq2Seq Model...")
for i in range(len(X_test)):
    current_X_test_sample = X_test[i:i+1]
    encoder_outputs_test, h_state, c_state = encoder_inference_model.predict(current_X_test_sample, verbose=0)
    decoder_input_data = np.zeros((1, 1, n_features))
    output_prediction, h_state, c_state = decoder_inference_model.predict(
        [decoder_input_data, h_state, c_state, encoder_outputs_test], verbose=0
    )
    attention_predictions.append(output_prediction.squeeze())

attention_predictions = np.array(attention_predictions)
print("Attention-based Seq2Seq Model predictions generated.")
print(f"Shape of LSTM predictions: {lstm_predictions.shape}")
print(f"Shape of Attention-based Seq2Seq predictions: {attention_predictions.shape}")

from sklearn.metrics import mean_squared_error, mean_absolute_error
import numpy as np

rmse_lstm = np.sqrt(mean_squared_error(y_test, lstm_predictions))
mae_lstm = mean_absolute_error(y_test, lstm_predictions)
rmse_attention = np.sqrt(mean_squared_error(y_test, attention_predictions))
mae_attention = mean_absolute_error(y_test, attention_predictions)

print("\n--- Model Performance Evaluation ---")
print(f"Baseline LSTM Model:")
print(f"  RMSE: {rmse_lstm:.4f}")
print(f"  MAE:  {mae_lstm:.4f}")

print(f"\nAttention-based Seq2Seq Model:")
print(f"  RMSE: {rmse_attention:.4f}")
print(f"  MAE:  {mae_attention:.4f}")

import matplotlib.pyplot as plt

print("Generating forecast comparison plots...")

for i in range(n_features):
    plt.figure(figsize=(12, 6))
    plt.plot(y_test[:, i], label='Actual Values', color='blue')
    plt.plot(lstm_predictions[:, i], label='LSTM Predictions', color='red', linestyle='--')
    plt.plot(attention_predictions[:, i], label='Attention Seq2Seq Predictions', color='green', linestyle=':')

    plt.title(f'Feature {i+1}: Actual vs. Predicted Values')
    plt.xlabel('Time Step')
    plt.ylabel('Scaled Differenced Value')
    plt.legend()
    plt.tight_layout()
    plt.show()

print("Forecast comparison plots generated.")